In [1]:
from selenium import webdriver # 브라우저 제어
from selenium.webdriver.common.by import By # 요소찾는 방법결정
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options # 브라우저 옵션 지정
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import re

In [3]:
# 엑셀 파일 읽기
df = pd.read_excel("캠핑장.xlsx")

# 'facltNm' 열을 리스트로 변환 (NaN 제거)
camping_list = df['facltNm'].dropna().tolist()

total_count = len(camping_list)
camping_list = camping_list[29:865]

print(f"총 캠핑장 개수: {len(camping_list)}")
print(f"예시 5개: {camping_list[:5]}")

총 캠핑장 개수: 836
예시 5개: ['덕두글램핑', '바다노을 캠핑장', '동화힐링캠프', '자연과우리캠핑장', '캠프아이엔지']


In [4]:
chrome_options = Options()
chrome_options.add_experimental_option('detach', True) # 브라우저 종료방지
driver = webdriver.Chrome(options=chrome_options)

# 웹페이지가 로딩될때가지 최대 10초 대기
driver.implicitly_wait(10)

# 브라우저 창 최대화
driver.maximize_window()

In [5]:
url = 'https://map.naver.com'
# 웹사이트 열기
driver.get(url)
time.sleep(5)

In [ ]:
for keyword in camping_list:
    try:
        # --- 검색어 입력 ---
        input_search = driver.find_element(By.CSS_SELECTOR, 'input.input_search')
        input_search.click()
        input_search.clear()
        input_search.send_keys(keyword)
        input_search.send_keys(Keys.ENTER)
        time.sleep(6)

        driver.switch_to.default_content()

        # --- searchIframe 확인 및 전환 ---
        iframe_list = driver.find_elements(By.ID, "searchIframe")
        if iframe_list:
            driver.switch_to.frame(iframe_list[0])
            time.sleep(6)

            # 💡 XPATH를 사용하여 이름이 일치하는 항목을 찾아 클릭
            try:
                # 텍스트가 정확히 keyword와 일치하는 요소를 XPATH로 찾기
                xpath_selector = f'//*[@id="_pcmap_list_scroll_container"]//span[text()="{keyword}"]'
                
                # XPath로 요소 찾기
                target_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, xpath_selector))
                )
                
                # 요소를 찾았으면 클릭
                driver.execute_script("arguments[0].click();", target_element)
                print(f"[정확한 결과 클릭 완료] {keyword}")
            
            except TimeoutException:
                print(f"[일치하는 결과 없음] {keyword}")
                driver.switch_to.default_content()
                continue
            
            time.sleep(6)
            driver.switch_to.default_content()

        else:
            print(f"[단일 결과 페이지] {keyword}")
            driver.switch_to.default_content()
            try:
                WebDriverWait(driver, 10).until(
                    EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
                )
            except TimeoutException:
                print(f"[단일결과 iframe 로딩 실패] {keyword}")
                continue

        # --- entryIframe 전환 및 리뷰 탭 클릭 (기존 코드와 동일) ---
        try:
            if not iframe_list:
                pass
            else:
                entry_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "entryIframe"))
                )
                driver.switch_to.frame(entry_iframe)
            
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located(
                        (By.CSS_SELECTOR, '#app-root div.place_fixed_maintab div div div div a')
                    )
                )
                tab_elements = driver.find_elements(
                    By.CSS_SELECTOR, '#app-root div.place_fixed_maintab div div div div a'
                )
                clicked = False
                for tab in tab_elements:
                    text = tab.text.strip()
                    if '리뷰' in text:
                        driver.execute_script("arguments[0].scrollIntoView(true);", tab)
                        driver.execute_script("arguments[0].click();", tab)
                        print(f"[리뷰 탭 접근 완료] {keyword}")
                        clicked = True
                        
                        # 💡 '더보기' 버튼이 사라질 때까지 반복해서 클릭하는 코드
                        more_button_xpath = '//*[@id="app-root"]/div/div/div[7]/div[3]/div/div/div/div[2]/div/a/span[contains(text(), "더보기")]'
                        try:
                            while True:
                                try:
                                    WebDriverWait(driver, 3).until(
                                        EC.presence_of_element_located((By.XPATH, more_button_xpath))
                                    )
                                    more_button = driver.find_element(By.XPATH, more_button_xpath)
                                    driver.execute_script("arguments[0].click();", more_button)
                                    time.sleep(1)
                                    print(f"[더보기 버튼 클릭 완료] {keyword}")
                                
                                except TimeoutException:
                                    print(f"[더보기 버튼 없음, 모든 키워드 로드됨] {keyword}")
                                    break

                        except Exception as e:
                            print(f"[더보기 버튼 클릭 실패] {keyword}, 에러: {e}")

                        # 💡 키워드 및 인원수 데이터 추출 로직 수정
                        keyword_data = []

                        keyword_list_element = None
                        
                        # 첫 번째 XPath로 시도
                        try:
                            keyword_list_xpath1 = '//*[@id="app-root"]/div/div/div[7]/div[3]/div[1]/div/div/div[2]/ul'
                            keyword_list_element = WebDriverWait(driver, 5).until(
                                EC.presence_of_element_located((By.XPATH, keyword_list_xpath1))
                            )
                        except TimeoutException:
                            print(f"[div[1] 경로에서 키워드 목록 찾기 실패] {keyword}")
                        
                        # 첫 번째 시도가 실패하면 두 번째 XPath로 시도
                        if not keyword_list_element:
                            try:
                                keyword_list_xpath2 = '//*[@id="app-root"]/div/div/div[7]/div[3]/div[2]/div/div/div[2]/ul'
                                keyword_list_element = WebDriverWait(driver, 5).until(
                                    EC.presence_of_element_located((By.XPATH, keyword_list_xpath2))
                                )
                            except TimeoutException:
                                print(f"[div[2] 경로에서 키워드 목록 찾기 실패] {keyword}")
                        
                        # 두 시도 모두 실패했을 경우
                        if not keyword_list_element:
                            print(f"[{keyword}] 키워드 목록을 찾을 수 없습니다.")
                            continue # 다음 캠핑장으로 넘어갑니다.

                        # 모든 키워드 항목(li 태그) 찾기
                        keyword_items = keyword_list_element.find_elements(By.XPATH, './li')

                        if not keyword_items:
                            print(f"[{keyword}] 키워드 데이터가 없습니다.")
                        else:
                            for item in keyword_items:
                                try:
                                    # 키워드 텍스트 추출 (기존과 동일)
                                    keyword_element = item.find_element(By.XPATH, './/span[1]')
                                    keyword_text = keyword_element.text.strip()
                                    
                                    # 💡 인원 수 텍스트를 가져오는 부분 수정
                                    # 인원수를 포함하는 부모 span 요소 선택
                                    count_element = item.find_element(By.XPATH, './/span[2]')
                                    
                                    # 부모 요소의 전체 텍스트를 가져와서 숫자만 추출
                                    count_text = count_element.text
                                    
                                    # 정규 표현식을 사용하여 숫자만 추출
                                    count_match = re.search(r'(\d+)', count_text)
                                    
                                    if count_match:
                                        count_number = int(count_match.group(1)) # 그룹 1의 값을 추출
                                    else:
                                        count_number = 0
                                        
                                    # 데이터 수집
                                    keyword_data.append({
                                        '캠핑장명': keyword,
                                        '키워드': keyword_text,
                                        '선택인원': count_number
                                    })
                                    
                                except NoSuchElementException:
                                    continue
                        
                        # CSV 파일로 저장
                        if keyword_data:
                            df = pd.DataFrame(keyword_data)
                            try:
                                with open('camping_keywords.csv', 'a', encoding='utf-8-sig', newline='') as f:
                                    df.to_csv(f, header=f.tell()==0, index=False)
                                print(f"[{keyword}] 키워드 데이터가 'camping_keywords.csv'에 저장되었습니다.")
                            except Exception as e:
                                print(f"CSV 저장 중 오류 발생: {e}")

                        time.sleep(5)
                        break
                if not clicked:
                    print(f"[리뷰 탭 없음] {keyword}")
                time.sleep(5)

            except TimeoutException:
                print(f"[리뷰 탭 로딩 실패] {keyword}")

        except NoSuchElementException:
            print(f"[entryIframe 없음] {keyword}")

        finally:
            driver.switch_to.default_content()
            time.sleep(5)

    except Exception as e:
        print(f"[검색 실패] {keyword}, 에러: {e}")


[정확한 결과 클릭 완료] 덕두글램핑
[리뷰 탭 접근 완료] 덕두글램핑
[더보기 버튼 클릭 완료] 덕두글램핑
[더보기 버튼 클릭 완료] 덕두글램핑
[더보기 버튼 클릭 완료] 덕두글램핑
[더보기 버튼 클릭 완료] 덕두글램핑
[더보기 버튼 클릭 완료] 덕두글램핑
[더보기 버튼 클릭 완료] 덕두글램핑
[더보기 버튼 없음, 모든 키워드 로드됨] 덕두글램핑
[div[1] 경로에서 키워드 목록 찾기 실패] 덕두글램핑
[덕두글램핑] 키워드 데이터가 'camping_keywords.csv'에 저장되었습니다.
[일치하는 결과 없음] 바다노을 캠핑장
[일치하는 결과 없음] 동화힐링캠프
[정확한 결과 클릭 완료] 자연과우리캠핑장
[리뷰 탭 접근 완료] 자연과우리캠핑장
[더보기 버튼 없음, 모든 키워드 로드됨] 자연과우리캠핑장
[div[1] 경로에서 키워드 목록 찾기 실패] 자연과우리캠핑장
[div[2] 경로에서 키워드 목록 찾기 실패] 자연과우리캠핑장
[자연과우리캠핑장] 키워드 목록을 찾을 수 없습니다.
[일치하는 결과 없음] 캠프아이엔지
[정확한 결과 클릭 완료] 엠와이힐링캠핑장
[리뷰 탭 접근 완료] 엠와이힐링캠핑장
[더보기 버튼 없음, 모든 키워드 로드됨] 엠와이힐링캠핑장
[div[1] 경로에서 키워드 목록 찾기 실패] 엠와이힐링캠핑장
[div[2] 경로에서 키워드 목록 찾기 실패] 엠와이힐링캠핑장
[엠와이힐링캠핑장] 키워드 목록을 찾을 수 없습니다.
[일치하는 결과 없음] 단양 별마루캠핑장
[정확한 결과 클릭 완료] 반시
[리뷰 탭 로딩 실패] 반시
[일치하는 결과 없음] 명마동 캠핑장
[정확한 결과 클릭 완료] 구리 토평 가족캠핑장
[리뷰 탭 접근 완료] 구리 토평 가족캠핑장
[더보기 버튼 없음, 모든 키워드 로드됨] 구리 토평 가족캠핑장
[div[1] 경로에서 키워드 목록 찾기 실패] 구리 토평 가족캠핑장
[div[2] 경로에서 키워드 목록 찾기 실패] 구리 토평 가족캠핑장
[구리 토평 가족캠핑장] 키워드 목록을 찾을 수 없습니다.
[일치하는 결과 없음] 낙동강레포츠밸